In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import random_split, Subset
from datetime import datetime
import numpy as np



# Getting the same results with train and train_manual_update
- Write torch.manual_seed(42) at the beginning of your notebook.
- Write torch.set_default_dtype(torch.double) at the beginning of your notebook to alleviate precision errors

In [2]:
torch.manual_seed(42)
torch.set_default_dtype(torch.double)
# Set device for the training
DEVICE_ = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Using device: {DEVICE_}")

Using device: cpu


# Tasks
Load, analyse and preprocess the CIFAR-10 dataset. Split it into 3
datasets: training, validation and test. Take a subset of these datasets
by keeping only 2 labels: cat and car

In [3]:
def load_cifar(train_val_split=0.9, data_path='../data/', preprocessor=None):
    """
    Rezize to 16x16 and normalize
    Load CIFAR-10 dataset and then filter for cars and cats
    """
    if preprocessor is None:
        preprocessor = transforms.Compose([
            transforms.Resize((16, 16)),  # Resize to get 768 input features
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    
    # Load full training and test datasets
    train_full = datasets.CIFAR10(data_path, train=True, download=True, transform=preprocessor)
    test_full = datasets.CIFAR10(data_path, train=False, download=True, transform=preprocessor)
    # Printing classes for finding cars and cats
    print("Classes in CIFAR-10 dataset:", train_full.classes)
    
    # Filter for cats (class 3) and cars(Automobile) (class 1) 
    def filter_ds(dataset, classes=[1, 3]):
        indices = [(i, label) for i, (_, label) in enumerate(dataset) if label in classes]
        # Remap labels to 0 and 1 for easy classing
        label_map = {classes[0]: 0, classes[1]: 1}
        indices = [(i, label_map[label]) for i, label in indices]
        subset = Subset(dataset, [i for i, _ in indices])
        return subset
    
    #Filtered
    train_filtered = filter_ds(train_full)
    test_filtered = filter_ds(test_full)
    
    # Split training data into train and validation
    train_size = int(len(train_filtered) * train_val_split)
    val_size = len(train_filtered) - train_size
    train_dataset, val_dataset = random_split(train_filtered, [train_size, val_size])
    
    return train_dataset, val_dataset, test_filtered


def compute_accuracy(model, loader):
 #TODO
 return 

In [5]:
load_cifar()

100%|██████████| 170M/170M [01:08<00:00, 2.48MB/s] 


Classes in CIFAR-10 dataset: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


(<torch.utils.data.dataset.Subset at 0x1af2940a990>,
 <torch.utils.data.dataset.Subset at 0x1af2939ebd0>)

Write a MyMLP class that implements a MLP in PyTorch (so only fully
connected layers) such that:
    
    - The input dimension is 768(= 16 ∗ 16 ∗ 3) and the output dimension is 2 (for the 2 classes).
    - The hidden layers have respectively 128 and 32 hidden units.
    - All activation functions are ReLU. The last layer has no activation function since the cross-entropy loss already includes a softmax activation
function.

In [4]:
class MyNet(nn.Module):
    #TODO

SyntaxError: incomplete input (2919787615.py, line 2)

Write a train(n_epochs, optimizer, model, loss_fn, train_loader) function that trains model for n_epochs epochs given an optimizer optimizer, a loss function loss_fn and a dataloader train_loader.

In [ ]:
# inspirasjon hentet fra https://github.com/simsam8/inf265_project1/blob/main/gradient_descent.ipynb
def train(n_epochs: int, optimizer: optim.Optimizer, model: nn.Module, loss_fn, train_loader, val_loader):
    #Tren modellen med en PyTorch optimizer
    print(f"Trener {model.__class__.__name__} med optimizer...")
    n_batches = len(train_loader)
    train_losses = []
    val_losses = []

    for epoch in range(1, n_epochs + 1):
        model.train()
        epoch_loss = 0.0
        #Treningsløkke
        for imgs, labels in train_loader:
            imgs = imgs.to(device=DEVICE_, dtype=torch.double)
            labels = labels.to(device=DEVICE_)
            
            #Fremoverpass og tapberegning
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            
            #Oppdater parametere og nullstill gradienter
            optimizer.step()
            optimizer.zero_grad()
            
            epoch_loss += loss.item()
        avg_train_loss = epoch_loss / n_batches
        train_losses.append(avg_train_loss)
        
        #Valideringsløkke
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs = imgs.to(DEVICE_, dtype=torch.double)
                labels = labels.to(DEVICE_)
                outputs = model(imgs)
                loss = loss_fn(outputs, labels)
                val_loss += loss.item()
        avg_val_loss = val_loss / n_batches
        val_losses.append(avg_val_loss)
        
        if epoch == 1 or epoch % 5 == 0:
            print(f"{datetime.now().time()} | Epoke {epoch}: Trenings-tap = {avg_train_loss:.5f}, Validerings-tap = {avg_val_loss:.5f}")
    
    return train_losses, val_losses



Write a similar function train manual_update that has no optimizer parameter, but a learning rate lr parameter instead and that manually updates each trainable parameter of model using equation (2). Do not forget to zero out all gradients after each iteration. 

Train 2 instances of MyMLP, one using train and the other using train_manual_update (use the same parameter values for both models). Compare their respective training losses. To get exactly the same results with both functions, see section 3.3

In [ ]:
def train_manual_update(n_epochs, model, loss_fn, train_loader, lr=1e-2, momentum_coeff=0., weight_decay=0.):
    